In [1]:
# Import your necessary package
import time
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [2]:
% matplotlib inline

In [3]:
seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(8)
np.random.seed(seed)
torch.manual_seed(seed)
dtype = np.float32

In [4]:
# Build and transform the dataset
kwargs = {"num_workers": 1, "pin_memory": True}
trains = datasets.MNIST('./data', train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                        ]))
tests = datasets.MNIST('./data', train=False,
                       transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                        ]))
batch_size = 200
train_loader = torch.utils.data.DataLoader(trains, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(tests, batch_size=batch_size, shuffle=True, **kwargs)

In [5]:
num_train = len(trains)
num_test = len(tests)
# Input dimension
input_dim = 784
# Output dimension
output_dim = 10
# Step size/Learning rate
lrate = 1e-2
# L2 regularization term
tau = 5e-2
# number of epoches for training
num_epochs = 10
print("Number of training instances: {}".format(num_train))
print("Number of test instances: {}".format(num_test))

Number of training instances: 60000
Number of test instances: 10000


In [6]:
# Define your Logistic Regression model here
class LogisticsRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticsRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        result = self.linear(x)
        return result

In [7]:
# Your training code here
model = LogisticsRegression(input_dim, output_dim)
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=lrate, weight_decay=tau)  

# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, num_train//batch_size, loss.data))

Epoch: [1/10], Step: [100/300], Loss: 0.6146
Epoch: [1/10], Step: [200/300], Loss: 0.4428
Epoch: [1/10], Step: [300/300], Loss: 0.5176
Epoch: [2/10], Step: [100/300], Loss: 0.4384
Epoch: [2/10], Step: [200/300], Loss: 0.3701
Epoch: [2/10], Step: [300/300], Loss: 0.4617
Epoch: [3/10], Step: [100/300], Loss: 0.3934
Epoch: [3/10], Step: [200/300], Loss: 0.3925
Epoch: [3/10], Step: [300/300], Loss: 0.4084
Epoch: [4/10], Step: [100/300], Loss: 0.4116
Epoch: [4/10], Step: [200/300], Loss: 0.3692
Epoch: [4/10], Step: [300/300], Loss: 0.4131
Epoch: [5/10], Step: [100/300], Loss: 0.3640
Epoch: [5/10], Step: [200/300], Loss: 0.3486
Epoch: [5/10], Step: [300/300], Loss: 0.4600
Epoch: [6/10], Step: [100/300], Loss: 0.3972
Epoch: [6/10], Step: [200/300], Loss: 0.4004
Epoch: [6/10], Step: [300/300], Loss: 0.4148
Epoch: [7/10], Step: [100/300], Loss: 0.3462
Epoch: [7/10], Step: [200/300], Loss: 0.2923
Epoch: [7/10], Step: [300/300], Loss: 0.3987
Epoch: [8/10], Step: [100/300], Loss: 0.3486
Epoch: [8/

In [8]:
# Your test code here
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the model on the 10000 test images: 91 %
